# Visites des pages contributions

Dans cette exploration, le but est de savoir, les conséquences sur le traffique qu'à eu la migration au nouveau format des pages contributions

Étape 1: Avoir le total des visites pour une contrib (générique + personnalisées) du 08/12/23 au 19/12/23

Étape 2: même chose du 12/01/24 au 23/01/24

## 1. Chargement des librairies

In [1]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)


# décommenter la période à tester

#query_before_start = "2023-12-08"
#query_before_end = "2023-12-19"

#query_after_start = "2024-01-12"
#query_after_end = "2024-01-23"


query_before_start = "2023-10-21"
query_before_end = "2023-12-06"

query_after_start = "2023-12-08"
query_after_end = "2024-01-23"

## 2. Récupération des queries sur elasticsearch

In [2]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY_BEFORE = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/contribution/" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": query_before_start,
                    "lte": query_before_end
               }
              }
            }
          ]
        }
      }
}


QUERY_AFTER = {
    "query": {
        "bool": { 
          "must": [
            {
              "prefix": {
                "url": "https://code.travail.gouv.fr/contribution/" 
              }
            },
            {
              "range": {
               "logfile": {
                    "gte": query_after_start,
                    "lte": query_after_end
               }
              }
            }
          ]
        }
      }
}

connection with ElasticSearch established with success


In [3]:
logs_before = es_connector.execute_query(QUERY_BEFORE, "logs-new")

  0%|          | 0/929662 [00:00<?, ?it/s]

In [4]:
logs_after = es_connector.execute_query(QUERY_AFTER, "logs-new")

  0%|          | 0/884011 [00:00<?, ?it/s]

In [ ]:
logs_before.count()


## 3. Compte pour chaque contributions du nombre de ligne

In [5]:
contribs_1 = [
    "est-il-obligatoire-davoir-un-contrat-de-travail-ecrit-et-signe",
    "quelle-est-la-duree-du-preavis-en-cas-de-demission",
    "quelle-est-la-duree-de-preavis-en-cas-de-depart-a-la-retraite",
    "quelles-sont-les-consequences-du-non-respect-du-preavis-par-le-salarie-ou-lemployeur",
    "quand-le-salarie-a-t-il-droit-a-une-prime-danciennete-quel-est-son-montant",
    "a-quelles-indemnites-peut-pretendre-un-salarie-qui-part-a-la-retraite",
    "les-conges-pour-evenements-familiaux",
    "quelles-sont-les-primes-prevues-par-la-convention-collective"
]

contribs_2 = [
    "la-periode-dessai-peut-elle-etre-renouvelee",
    "quelle-est-la-duree-de-preavis-en-cas-de-licenciement"
    
]

contribs = contribs_2

In [6]:
### Avant

In [12]:
nombre_de_lignes_par_chaine = []

for c in contribs:
    count = logs_before['url'].apply(lambda x: c in x).sum()
    nombre_de_lignes_par_chaine.append(count)


print(f"AVANT", query_before_start, query_before_end)
print(f"Contrib | Visites")
for chaine, count in zip(contribs, nombre_de_lignes_par_chaine):
    print(f" {chaine} | {count}")


AVANT 2023-10-21
Contrib | Visites
 la-periode-dessai-peut-elle-etre-renouvelee | 14002
 quelle-est-la-duree-de-preavis-en-cas-de-licenciement | 17296


In [8]:
### Après

In [13]:
nombre_de_lignes_par_chaine = []

for c in contribs:
    count = logs_after['url'].apply(lambda x: c in x).sum()
    nombre_de_lignes_par_chaine.append(count)

print(f"APRES", query_after_start, query_after_end)
print(f"Contrib | Visites")
for chaine, count in zip(contribs, nombre_de_lignes_par_chaine):
    print(f"{chaine} | {count}")


APRES 2023-12-08 2024-01-23
Contrib | Visites
la-periode-dessai-peut-elle-etre-renouvelee | 35207
quelle-est-la-duree-de-preavis-en-cas-de-licenciement | 20379


In [10]:
filtered_logs = logs_before[logs_before['url'].str.contains('quelle-est-la-duree-de-preavis-en-cas-de-licenciement')]

filtered_logs.groupby(by=["url"]).agg({"idVisit": "nunique"}).sort_values(by="idVisit", ascending=False).head(1000)


,idVisit
url,
https://code.travail.gouv.fr/contribution/quelle-est-la-duree-de-preavis-en-cas-de-licenciement,7340
https://code.travail.gouv.fr/contribution/650-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,974
https://code.travail.gouv.fr/contribution/2941-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,837
https://code.travail.gouv.fr/contribution/16-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,545
https://code.travail.gouv.fr/contribution/413-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,430
https://code.travail.gouv.fr/contribution/2609-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,293
https://code.travail.gouv.fr/contribution/3209-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,189
https://code.travail.gouv.fr/contribution/1501-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,156
https://code.travail.gouv.fr/contribution/292-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,150


In [11]:
filtered_logs = logs_after[logs_after['url'].str.contains('quelle-est-la-duree-de-preavis-en-cas-de-licenciement')]
filtered_logs.groupby(by=["url"]).agg({"idVisit": "nunique"}).sort_values(by="idVisit", ascending=False)

,idVisit
url,
https://code.travail.gouv.fr/contribution/quelle-est-la-duree-de-preavis-en-cas-de-licenciement,3764
https://code.travail.gouv.fr/contribution/3248-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,989
https://code.travail.gouv.fr/contribution/16-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,433
https://code.travail.gouv.fr/contribution/650-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,376
https://code.travail.gouv.fr/contribution/2941-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,345
...,...
"https://code.travail.gouv.fr/contribution/2609-quelle-est-la-duree-de-preavis-en-cas-de-licenciement#:~:text=En cas de licenciement, la,ou non) : 3 mois.",1
https://code.travail.gouv.fr/contribution/2542-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,1
https://code.travail.gouv.fr/contribution/1635-quelle-est-la-duree-de-preavis-en-cas-de-licenciement,1
